# first, we are loading some data

In [1]:
import xgboost as xgb
import cPickle as pickle
from sklearn.metrics import roc_curve, auc
import numpy as np

def load_data(name):
    f=open(name+'.pk1','rb')
    tmp=pickle.load(f)
    f.close()
    return tmp

sample_data, sample_target, sample_cv = \
    load_data('sample_data'), load_data('sample_target'), load_data('sample_cv')

# and setting xgboost params

In [2]:
xgb_params= {'eta' :0.5, 'max_depth':6, 'subsample':0.7, 'nthread':32,'eval_metrics':'auc','seed':27}

# now making CV with xgb.cv 

In [3]:
xgall = xgb.DMatrix(sample_data, label=sample_target)

cv_res=xgb.cv(xgb_params,xgall,100, 3, verbose_eval= 10,metrics='auc', folds=sample_cv )

[0]	cv-test-auc:0.735845571429+0.0167816785799	cv-train-auc:0.737591714286+0.00667894561213
[10]	cv-test-auc:0.874163571429+0.0237914165125	cv-train-auc:0.883316142857+0.00478725013309
[20]	cv-test-auc:0.886021571429+0.0236185393697	cv-train-auc:0.897613428571+0.00578312488087
[30]	cv-test-auc:0.889775285714+0.0235858872919	cv-train-auc:0.902816428571+0.00633667485712
[40]	cv-test-auc:0.892011857143+0.0245071014339	cv-train-auc:0.905576857143+0.00634646579778
[50]	cv-test-auc:0.892436714286+0.0248738822848	cv-train-auc:0.906705571429+0.00632565919449
[60]	cv-test-auc:0.893096714286+0.0258509233751	cv-train-auc:0.907717142857+0.00708955066939
[70]	cv-test-auc:0.893982571429+0.0261236180433	cv-train-auc:0.908896857143+0.00732495215837
[80]	cv-test-auc:0.894836+0.0262578288951	cv-train-auc:0.909939571429+0.00710310607827
[90]	cv-test-auc:0.895363285714+0.0265216606823	cv-train-auc:0.910703714286+0.0070735471343


# wow, we got nice AUC

In [4]:
print 'our auc on test is:', cv_res.values[99][0]

our auc on test is: 0.895906714286


# now let's make CV using python

In [5]:
xgbooster = xgb.Booster()

res=[]

for a,b in sample_cv:
    xgtrain = xgb.DMatrix(sample_data[a], label=sample_target[a])
    xgbtest = xgb.DMatrix(sample_data[b], label=sample_target[b])
    
    bst = xgb.train( xgb_params, xgtrain, 100)
    pred=bst.predict(xgbtest)
    
    fpr, tpr, _ = roc_curve(sample_target[b], pred)
    roc_auc = auc(fpr, tpr)

    res.append(roc_auc)

# and getting completely different results!!!

In [6]:
print 'results:', res, '\n'
print 'wtf! mean AUC is ', np.mean(res)

results: [0.78109900929252929, 0.8434556366940299, 0.86579716116771199, 0.80723688722001374, 0.56541167436109074, 0.79476800572814488, 0.77813671313453703] 

wtf! mean AUC is  0.776557869657
